In [ ]:
from pathlib import Path
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np

from src.utils.paths import load_config, get_data_paths, resolve_path

cfg = load_config('config/default.yaml')
paths = get_data_paths(cfg)
images_dir = paths['processed_images']
masks_dir = paths['processed_masks']

image_paths = sorted(images_dir.glob('*.png'))
mask_paths = sorted(masks_dir.glob('*.png'))
print(f"Found {len(image_paths)} images, {len(mask_paths)} masks")


In [ ]:
def load_pair(idx):
    img_path = image_paths[idx]
    mask_path = masks_dir / (img_path.stem + '.png')
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(str(mask_path), cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[:, :, 0]
    return img, mask

n_samples = min(8, len(image_paths))
indices = random.sample(range(len(image_paths)), n_samples)

fig, axes = plt.subplots(n_samples, 3, figsize=(9, 3 * n_samples))
for row, idx in enumerate(indices):
    img, mask = load_pair(idx)
    axes[row, 0].imshow(img)
    axes[row, 0].set_title(f'Image {idx}')
    axes[row, 0].axis('off')

    axes[row, 1].imshow(mask, cmap='tab20')
    axes[row, 1].set_title('Mask')
    axes[row, 1].axis('off')

    overlay = img.copy()
    colored = cv2.applyColorMap((mask.astype(np.uint8) * 30) % 256, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img, 0.6, colored, 0.4, 0)
    axes[row, 2].imshow(overlay)
    axes[row, 2].set_title('Overlay')
    axes[row, 2].axis('off')

plt.tight_layout()
